# Common DataFrames and SPARK SQL operations

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://5fcc63a3e7cf:4041
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1656339350585)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@37b9a802


In [3]:
// importamos las funciones
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._


In [4]:
// importamos dos archivos, uno ocn informacion sobre aeropuestos y otro sobre vuelos

## Creamos los dataframes

In [6]:
val airports = spark.read
    .option("header","true")
    .option("inferSchema","true")
    .option("delimiter","\t")
    .csv("airport-codes-na.txt")

// Creamos vista temporal
airports.createOrReplaceTempView("airports_na")

airports: org.apache.spark.sql.DataFrame = [City: string, State: string ... 2 more fields]


In [8]:
spark.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |airports_na|       true|
+---------+-----------+-----------+



In [9]:
airports.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- IATA: string (nullable = true)



In [10]:
val delays = spark.read
    .option("header","true")
    .option("inferSchema","true")
    .csv("departuredelays.csv")
delays.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



delays: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [11]:
delays.createOrReplaceTempView("delays_na")

In [14]:
spark.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |airports_na|       true|
|         |  delays_na|       true|
+---------+-----------+-----------+



In [41]:
val foo = delays.filter(
    expr("""origin == 'SEA' and destination = 'SFO' and delay > 0  and date like '10111%' """))
foo.createOrReplaceTempView("foo")

foo: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: int, delay: int ... 3 more fields]


In [17]:
delays.show(2)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
+-------+-----+--------+------+-----------+
only showing top 2 rows



In [42]:
foo.show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011100|    2|     590|   SEA|        SFO|
|1011125|   11|     590|   SEA|        SFO|
+-------+-----+--------+------+-----------+



In [48]:
foo.createOrReplaceTempView("foo")

spark.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |airports_na|       true|
|         |  delays_na|       true|
|         |        foo|       true|
+---------+-----------+-----------+



## Comprobacion

In [44]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



In [49]:
spark.sql("SELECT * FROM delays_na LIMIT 10").show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+



In [50]:
spark.sql("SELECT * FROM foo").show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011100|    2|     590|   SEA|        SFO|
|1011125|   11|     590|   SEA|        SFO|
+-------+-----+--------+------+-----------+



## Unions

In [51]:
val bar = delays.union(foo)
bar.createOrReplaceTempView("bar")

bar: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [date: int, delay: int ... 3 more fields]


In [52]:
bar.filter("""origin == 'SEA' and destination = 'SFO' and delay > 0  and date like '10111%' """).show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011100|    2|     590|   SEA|        SFO|
|1011125|   11|     590|   SEA|        SFO|
|1011100|    2|     590|   SEA|        SFO|
|1011125|   11|     590|   SEA|        SFO|
+-------+-----+--------+------+-----------+



In [54]:
spark.sql("""
    select *
    from bar
    where origin = 'SEA' and destination = 'SFO' and delay>0 and date like '10111%'
    """).show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011100|    2|     590|   SEA|        SFO|
|1011125|   11|     590|   SEA|        SFO|
|1011100|    2|     590|   SEA|        SFO|
|1011125|   11|     590|   SEA|        SFO|
+-------+-----+--------+------+-----------+



## JOINS

In [57]:
foo.join(airports.as("air"),
        col("air.IATA") === col("origin")).show()

+-------+-----+--------+------+-----------+-------+-----+-------+----+
|   date|delay|distance|origin|destination|   City|State|Country|IATA|
+-------+-----+--------+------+-----------+-------+-----+-------+----+
|1011100|    2|     590|   SEA|        SFO|Seattle|   WA|    USA| SEA|
|1011125|   11|     590|   SEA|        SFO|Seattle|   WA|    USA| SEA|
+-------+-----+--------+------+-----------+-------+-----+-------+----+



## Windowing

In [67]:
spark.sql("drop table if exists departureDelaysWindow")

res61: org.apache.spark.sql.DataFrame = []


In [70]:
 val df = delays_na.select("origin", "destination", expr(sum(col("delay")))
                           .where(col("origin") === 'SEA')

<console>: 3: error: unclosed character literal (or use " for string literal "SEA")

In [72]:
spark.sql("""create or replace temp view departureDelaysWindow as
             select origin, destination, sum(delay) as totaldelays
             from delays_na
             where origin in ('SEA','SFO','JFK') and destination in ('SEA','SFO','JFK','DEN','LAX','ATL')
             group by origin, destination
             """)

spark.sql("select * from departureDelaysWindow").show()

+------+-----------+-----------+
|origin|destination|totaldelays|
+------+-----------+-----------+
|   JFK|        SFO|      35619|
|   JFK|        DEN|       4315|
|   JFK|        ATL|      12141|
|   JFK|        SEA|       7856|
|   JFK|        LAX|      35755|
|   SEA|        LAX|       9359|
|   SFO|        DEN|      18688|
|   SFO|        SEA|      17080|
|   SEA|        SFO|      22293|
|   SFO|        ATL|       5091|
|   SEA|        DEN|      13645|
|   SEA|        ATL|       4535|
|   SFO|        JFK|      24100|
|   SFO|        LAX|      40798|
|   SEA|        JFK|       4667|
+------+-----------+-----------+



In [76]:
spark.sql("""select origin, destination, totaldelays, rank
             from ( select origin, destination, totaldelays, dense_rank()
                    over (partition by origin order by totaldelays desc) as rank
                    from departureDelaysWindow
                  ) t
             where rank <= 3
          """).show()

+------+-----------+-----------+----+
|origin|destination|totaldelays|rank|
+------+-----------+-----------+----+
|   JFK|        LAX|      35755|   1|
|   JFK|        SFO|      35619|   2|
|   JFK|        ATL|      12141|   3|
|   SEA|        SFO|      22293|   1|
|   SEA|        DEN|      13645|   2|
|   SEA|        LAX|       9359|   3|
|   SFO|        LAX|      40798|   1|
|   SFO|        JFK|      24100|   2|
|   SFO|        DEN|      18688|   3|
+------+-----------+-----------+----+



In [79]:
spark.sql("""select origin, destination, totaldelays, dense_rank()
             over (partition by origin order by totaldelays desc) as rank
             from departureDelaysWindow""").show()

+------+-----------+-----------+----+
|origin|destination|totaldelays|rank|
+------+-----------+-----------+----+
|   JFK|        LAX|      35755|   1|
|   JFK|        SFO|      35619|   2|
|   JFK|        ATL|      12141|   3|
|   JFK|        SEA|       7856|   4|
|   JFK|        DEN|       4315|   5|
|   SEA|        SFO|      22293|   1|
|   SEA|        DEN|      13645|   2|
|   SEA|        LAX|       9359|   3|
|   SEA|        JFK|       4667|   4|
|   SEA|        ATL|       4535|   5|
|   SFO|        LAX|      40798|   1|
|   SFO|        JFK|      24100|   2|
|   SFO|        DEN|      18688|   3|
|   SFO|        SEA|      17080|   4|
|   SFO|        ATL|       5091|   5|
+------+-----------+-----------+----+



## Modifications

In [80]:
foo.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011100|    2|     590|   SEA|        SFO|
|1011125|   11|     590|   SEA|        SFO|
+-------+-----+--------+------+-----------+



### Añadir nueva columna 

In [81]:
val foo2 = foo.withColumn("status", expr("""case when delay<=10 then 'on-time'
                                               else 'delayed'
                                            end"""))

foo2.show()

+-------+-----+--------+------+-----------+-------+
|   date|delay|distance|origin|destination| status|
+-------+-----+--------+------+-----------+-------+
|1011100|    2|     590|   SEA|        SFO|on-time|
|1011125|   11|     590|   SEA|        SFO|delayed|
+-------+-----+--------+------+-----------+-------+



foo2: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 4 more fields]


### Eliminar columna

In [82]:
val foo3 = foo2.drop("delay")
foo3.show()

+-------+--------+------+-----------+-------+
|   date|distance|origin|destination| status|
+-------+--------+------+-----------+-------+
|1011100|     590|   SEA|        SFO|on-time|
|1011125|     590|   SEA|        SFO|delayed|
+-------+--------+------+-----------+-------+



foo3: org.apache.spark.sql.DataFrame = [date: int, distance: int ... 3 more fields]


### Cambiar nombre a la columna

In [84]:
val foo4 = foo2.withColumnRenamed("delay","retraso")
foo4.show()

+-------+-------+--------+------+-----------+-------+
|   date|retraso|distance|origin|destination| status|
+-------+-------+--------+------+-----------+-------+
|1011100|      2|     590|   SEA|        SFO|on-time|
|1011125|     11|     590|   SEA|        SFO|delayed|
+-------+-------+--------+------+-----------+-------+



foo4: org.apache.spark.sql.DataFrame = [date: int, retraso: int ... 4 more fields]


### Pivoting

In [86]:
spark.sql("""select destination, cast(substring(date,0,2) as int) as month, delay
             from delays_na 
             where origin = 'SEA'""").show()

+-----------+-----+-----+
|destination|month|delay|
+-----------+-----+-----+
|        ORD|   10|   92|
|        JFK|   10|   -7|
|        DFW|   10|   -5|
|        MIA|   10|   -3|
|        DFW|   10|   -3|
|        DFW|   10|    1|
|        ORD|   10|  -10|
|        DFW|   10|   -6|
|        DFW|   10|   -2|
|        ORD|   10|   -3|
|        ORD|   10|    0|
|        DFW|   10|   23|
|        DFW|   10|   36|
|        ORD|   10|  298|
|        JFK|   10|    4|
|        DFW|   10|    0|
|        MIA|   10|    2|
|        DFW|   10|    0|
|        DFW|   10|    0|
|        ORD|   10|   83|
+-----------+-----+-----+
only showing top 20 rows



In [89]:
spark.sql("""SELECT * 
             FROM ( SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
                    FROM delays_na
                    WHERE origin = 'SEA'
                   )
             PIVOT (
                    CAST(AVG(delay) AS DECIMAL(4, 2)) AS AvgDelay, MAX(delay) AS MaxDelay
                         FOR month IN (10 OCT, 11 NOV)
                    )
            ORDER BY destination""").show()

+-----------+------------+------------+------------+------------+
|destination|OCT_AvgDelay|OCT_MaxDelay|NOV_AvgDelay|NOV_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       12.00|          27|        8.25|          21|
|        ANC|        9.81|         149|        3.58|         106|
|        ATL|       18.74|         397|        7.89|         201|
|        AUS|       11.44|          50|        1.10|          31|
|        BOS|       16.19|         110|        7.81|         110|
|        BUR|       -4.67|           4|       -1.90|          54|
|        CLE|       16.00|          27|        null|        null|
|        CLT|        4.59|          41|        2.47|          33|
|        COS|       12.56|          82|        3.90|          42|
|        CVG|       -0.50|           4|        null|        null|
|        DCA|        0.50|          50|       -5.26|           2|
|        DEN|       26.99|         425|        7.69|         120|
|        D